# [StackSample: 10% of Stack Overflow Q&A EDA](https://www.kaggle.com/datasets/stackoverflow/stacksample?resource=download)

In [23]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup
questions_raw = pd.read_csv("./data/Questions.csv", encoding="ISO-8859-1")
answers_raw = pd.read_csv("./data/Answers.csv", encoding="ISO-8859-1")

In [25]:
tags_raw = pd.read_csv("./data/Tags.csv", encoding="ISO-8859-1", dtype={'Tags':'string'})

## Introductory remarks

Target labels

These needs to be group by id

ID is connected to questions csv id column

In [26]:
tags_raw.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


Wondering if answers bodies are useful to train the model?

It might be as answer bodies with high scores will have information corresponding to the subject tag

Answers might even have much higher correlation to the tag as they probably use more correct language, as the questioners are most likely quite novice programmers in this case

These should be grouped by parentId and which is connected to questions csv id column

We should also throw away answers below a certain Score which we need to figure out what the breakpoint is, I know tho that sometimes a really good answer does not get points as the question is not very common and points can only be given by other frequent answer posting people with higher scores

CreationDate, Id should be tossed

One intreating EDA question might be to group on OwnerUserId and check high scoring players as those people will most likely give very good answers connected to the tags

In [27]:
answers_raw.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


This is the main data frame and something that might be interesting to check out is how often titles contain the corresponding tag

We should also throw away questions that have lower scores as they might not have very good content that might not even relate to the tags, so this thresh hold should also be figured out

In [28]:
questions_raw.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


## Data cleaning

### Making tags into lists for each Id

In [29]:
type(tags_raw.iloc[0][1])

str

In [30]:
tag_lists = tags_raw.groupby('Id')['Tag'].apply(lambda tags: [tag for tag in tags])
tag_lists.head(1)

Id
80    [flex, actionscript-3, air]
Name: Tag, dtype: object

### Cleaning bodies

We need to clean all bodies from the html tags

I'm going to "steal" parts of what these two people did for their EDA's

- https://www.kaggle.com/code/niyamatalmass/texts-summarizing-with-the-help-of-spacy

- https://www.kaggle.com/code/miljan/predicting-tags-for-stackoverflow

This would not be applicable in a real world scenario but we only have 2 weeks to do this assigment

In [31]:
answers_dropped_columns = answers_raw[['ParentId', 'Score', 'Body', 'OwnerUserId']].rename(columns={'ParentId':'Id', 'OwnerUserId': 'AnswerUserId'})
answers_dropped_columns.head(1)

,Id,Score,Body,AnswerUserId
0,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers...",61.0


In [38]:
answers_cleaned_html = answers_dropped_columns.drop(columns=['Body']).join(answers_dropped_columns['Body'].apply(lambda x: BeautifulSoup(x).get_text()))
answers_cleaned_html.head(1)

,Id,Score,AnswerUserId,Body
0,90,13,61.0,Version Control with Subversion\nA very good r...


In [46]:
questions_dropped_columns = questions_raw[['Id', 'Title' , 'Score', 'Body', 'OwnerUserId']].rename(columns={'OwnerUserId': 'QuestionerUserId'})
questions_dropped_columns.head(1)

,Id,Title,Score,Body,QuestionerUserId
0,80,SQLStatement.execute() - multiple queries in o...,26,<p>I've written a database generation script i...,26.0


In [47]:
questions_cleaned_html = questions_dropped_columns.drop(columns=['Body']).join(questions_dropped_columns['Body'].apply(lambda x: BeautifulSoup(x).get_text()))
questions_cleaned_html.head(1)

,Id,Title,Score,QuestionerUserId,Body
0,80,SQLStatement.execute() - multiple queries in o...,26,26.0,I've written a database generation script in S...


* Note doing the beautiful soup transformations took 20 minutes to complete

In [42]:
for body in answers_cleaned_html.sample(10)['Body']:
    print('\n')
    print(body)



You use eclipses' ability to create a WAR file, just like you would for Java.  Or a jar file, just like you would for Java.  Because, at run time it is java.
No more magic needed.



Fixed it with "#{row["Price"].to_f}".
Thanks!



An RDBMS can normally return the result of a query with IN in equal or less time than it takes to execute two queries. 
If there is no index on column Y, then a full table scan is required. With two queries, two table scans will be performed instead of one.
If there is an index, then the single value in the WHERE clause, or the values in the IN list, are used one at a time to look up the index. When some rows are found for one of the values in the IN list, they are added to the returned result. 
So it is better to use the IN predicate from the performance point of view.
When Y represents a column with unique values, then it is easy to decompose the result. Otherwise, there is slightly more work.



This is not how HTTP and HTML work. A first request is mad

In [65]:
for title, body in questions_cleaned_html[['Title', 'Body']].sample(10).values:
    print(title)
    print(body)

Auto start print html page using javascript
Is there anyway to automatically run javascript:window.print() when the page finishes loading? 

iOS Background Thread blocking UI when printing to console
I have to retrieve a list of users and user data from Firebase. The data is relatively large so I used a background thread to retrieve it using the following code:
    dispatch_async(dispatch_get_global_queue(DISPATCH_QUEUE_PRIORITY_BACKGROUND, 0), ^{
        FIRDatabaseReference *ref = [[FIRDatabase database] reference];
        [[ref child:@"users"] observeSingleEventOfType:FIRDataEventTypeValue withBlock:^(FIRDataSnapshot * _Nonnull snapshot) {
            for (FIRDataSnapshot *userSnapshot in [snapshot children]) {
                NSLog(@"key %@ and value %@", userSnapshot.key, userSnapshot.value);
            }
        }];
    });

The problem occurs when the block starts executing. As the output is printed to the console, any user interaction with the application is blocked. I have t

## EDA for making of data-splitting-and-cleaning.ipynb

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from re import findall, sub

STACK_OVERFLOW_CODE_BLOCK_RE = r"<pre><code>([\s\S]*?)<\/code><\/pre>"


def extract_code_blocks_from_body(body):
    code_block = findall(STACK_OVERFLOW_CODE_BLOCK_RE, body)
    code_block = " ".join(code_block)
    if code_block == "":
        return None
    return code_block


def remove_code_blocks_from_body(body):
    new_body = sub(STACK_OVERFLOW_CODE_BLOCK_RE, "", body)
    return new_body


def remove_new_line(text):
    return text.replace("\n", " ")


In [2]:
questions_raw = pd.read_csv("./data/Questions.csv", usecols=["Id", "Body", "Title"], encoding="ISO-8859-1").rename(
    columns={"Id": "id", "Body": "body", "Title": "title"}
)
tags_raw = pd.read_csv("./data/Tags.csv", encoding="ISO-8859-1", dtype={"Tags": "string"}).rename(
    columns={"Id": "id", "Tag": "tag"}
)

In [3]:
code_blocks = questions_raw["body"].apply(lambda body: extract_code_blocks_from_body(body)).rename("code")
code_blocks.head(1)

0    Create Table tRole (\n      roleID integer Pri...
Name: code, dtype: object

In [4]:
body_without_code_blocks = questions_raw["body"].apply(lambda body: remove_code_blocks_from_body(body)).rename("desc")
body_without_code_blocks.head(1)

0    <p>I've written a database generation script i...
Name: desc, dtype: object

In [5]:
questions_body_split = questions_raw.drop(columns=["body"]).join(body_without_code_blocks).join(code_blocks)
questions_body_split.head(1)

,id,title,desc,code
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,Create Table tRole (\n roleID integer Pri...


In [6]:
questions_cleaned_html = questions_body_split.drop(columns=["desc"]).join(
    questions_body_split["desc"].apply(lambda body: BeautifulSoup(body).get_text())
)

questions_cleaned_html.head(1)

,id,title,code,desc
0,80,SQLStatement.execute() - multiple queries in o...,Create Table tRole (\n roleID integer Pri...,I've written a database generation script in S...


In [7]:
questions_cleaned = questions_cleaned_html.drop(columns=["desc"]).join(
    questions_cleaned_html["desc"].apply(lambda body: remove_new_line(body))
)
questions_cleaned.head(1)

,id,title,code,desc
0,80,SQLStatement.execute() - multiple queries in o...,Create Table tRole (\n roleID integer Pri...,I've written a database generation script in S...


In [8]:
tag_lists = tags_raw.groupby("id")["tag"].apply(lambda tags: [tag for tag in tags]).rename('tags')
tag_lists.head(1)

id
80    [flex, actionscript-3, air]
Name: tags, dtype: object

In [61]:
tags_combined_with_questions = questions_cleaned.set_index("id").join(tag_lists).reset_index()
tags_combined_with_questions.head(1)

,id,title,code,desc,tags
0,80,SQLStatement.execute() - multiple queries in o...,Create Table tRole (\n roleID integer Pri...,I've written a database generation script in S...,"[flex, actionscript-3, air]"


In [50]:
tags_combined_with_questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264216 entries, 0 to 1264215
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   id      1264216 non-null  int64 
 1   title   1264216 non-null  object
 2   code    840834 non-null   object
 3   desc    1264216 non-null  object
 4   tags    1264216 non-null  object
dtypes: int64(1), object(4)
memory usage: 48.2+ MB


In [20]:
tags_with_less_than_50_samples = pd.DataFrame(tags_raw["tag"].value_counts()).query("tag < 50").index

In [51]:
tag_counts = tags_combined_with_questions.join(
    tags_combined_with_questions["tags"].apply(lambda tags: len(tags)).rename("tag_counts")
)
tag_counts.head(1)

,id,title,code,desc,tags,tag_counts
0,80,SQLStatement.execute() - multiple queries in o...,Create Table tRole (\n roleID integer Pri...,I've written a database generation script in S...,"[flex, actionscript-3, air]",3


In [47]:
rows_that_only_contain_a_tag_that_has_less_than_50_samples = tag_counts.query("tag_counts == 1")[
    tag_counts.query("tag_counts == 1").apply(lambda row: row[4][0] in tags_with_less_than_50_samples, axis=1).values
]
rows_that_only_contain_a_tag_that_has_less_than_50_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7099 entries, 37 to 1264009
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          7099 non-null   int64 
 1   title       7099 non-null   object
 2   code        2769 non-null   object
 3   desc        7099 non-null   object
 4   tags        7099 non-null   object
 5   tag_counts  7099 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 388.2+ KB


In [57]:
rows_that_only_contain_a_tag_that_has_less_than_50_samples["tags"].apply(lambda tags: tags[0]).to_csv("./thrown_away_tags_list.csv")

In [65]:
questions_without_rows_that_has_less_a_that_that_has_less_that_50_samples = tags_combined_with_questions.drop(
    rows_that_only_contain_a_tag_that_has_less_than_50_samples.index
)

questions_without_rows_that_has_less_a_that_that_has_less_that_50_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1257117 entries, 0 to 1264215
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   id      1257117 non-null  int64 
 1   title   1257117 non-null  object
 2   code    838065 non-null   object
 3   desc    1257117 non-null  object
 4   tags    1257117 non-null  object
dtypes: int64(1), object(4)
memory usage: 57.5+ MB


In [67]:
def remove_elements_from_list(*, removing_elements, target_list):
    return [element for element in target_list if element not in removing_elements]
a = ['test', 'test2', 'test3']
b = ['test', 'test2']
remove_elements_from_list(removing_elements=b, target_list=a)

['test3']

In [73]:
questions_removed_tags_with_less_than_50_samples = (
    questions_without_rows_that_has_less_a_that_that_has_less_that_50_samples.drop(columns=["tags"]).join(
        questions_without_rows_that_has_less_a_that_that_has_less_that_50_samples["tags"].apply(
            lambda tags: remove_elements_from_list(removing_elements=tags_with_less_than_50_samples, target_list=tags)
        )
    )
)
questions_removed_tags_with_less_than_50_samples.head(1)

,id,title,code,desc,tags
0,80,SQLStatement.execute() - multiple queries in o...,Create Table tRole (\n roleID integer Pri...,I've written a database generation script in S...,"[flex, actionscript-3, air]"


In [75]:
questions_cleaned = questions_removed_tags_with_less_than_50_samples.drop(columns=['id'])
questions_cleaned.head()

,title,code,desc,tags
0,SQLStatement.execute() - multiple queries in o...,Create Table tRole (\n roleID integer Pri...,I've written a database generation script in S...,"[flex, actionscript-3, air]"
1,Good branching and merging tutorials for Torto...,None,Are there any really good tutorials explaining...,"[svn, tortoisesvn, branch, branching-and-merging]"
2,ASP.NET Site Maps,None,Has anyone got experience creating SQL-based A...,"[sql, asp.net, sitemap]"
3,Function for creating color wheels,None,This is something I've pseudo-solved many time...,"[algorithm, language-agnostic, colors]"
4,Adding scripting functionality to .NET applica...,ICard Cards[current] = new MyGame.CardLibrary....,I have a little game written in C#. It uses a ...,"[c#, .net, scripting, compiler-construction]"
